In [1]:
from langchain_ollama import ChatOllama

# Initialize LLMs
llm = ChatOllama(
    model="gemma3:1b", 
    base_url="http://localhost:11434", 
    temperature=0.5
)

tc_llm = ChatOllama(
    model="gemma3:1b", 
    base_url="http://localhost:11434", 
    temperature=0.5
)

In [2]:
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate

@tool
def generate_test_case(user_story: str) -> str:
    """Generate test case in Gherkin format from a user story"""
    
    prompt_template = PromptTemplate.from_template(
        """
        You are a QA Automation Engineer. 
        Your task is to convert the following user story into at ONLY 1 test case, in Gherkin BDD style format.
        Include combinations of valid, invalid, edge case and alternative flow scenarios.
        
        User Story: {user_story}
        
        Format your response as:
        Feature: [Feature name]
        
        Scenario: [Scenario name]
            Given [precondition]
            When [action]
            Then [expected result]
        
        Scenario: [Another scenario if applicable]
            Given [precondition]
            When [action]
            Then [expected result]
        """
    )
    
    prompt = prompt_template.format(user_story=user_story)
    response = tc_llm.invoke(prompt)
    
    # Extract content from the response
    if hasattr(response, 'content'):
        return response.content
    else:
        return str(response)

In [3]:
from langchain.agents import initialize_agent, AgentType


# Initialize agent with a simpler agent type
agent = initialize_agent(
    tools=[generate_test_case],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Changed from STRUCTURED_CHAT
    verbose=True,
    handle_parsing_errors=True
)

# User story input
user_story_input = """
As a user, I want to reset my password using a link sent to my email, so that I can
regain access to my account if I forgot my password.
"""

# Execute the agent
try:
    results = agent.invoke({
        "input": f"Create BDD test cases for this user story: {user_story_input}"
    })
    print("Results:")
    print(results['output'])
except Exception as e:
    print(f"Error occurred: {e}")
    # Fallback: call the tool directly
    print("\nFalling back to direct tool call:")
    direct_result = generate_test_case(user_story_input.strip())
    print(direct_result)


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_23872\2669201702.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
The user story describes a feature where a user can reset their password. The goal is to allow the user to regain access to their account if they forget their password. We need to create test cases to cover this functionality.

Action: generate_test_case
Action Input: As a user, I want to reset my password using a link sent to my email, so that I can
regain access to my account if I forgot my password.
Observation: Okay, here's a single Gherkin BDD test case for the user story: "As a user, I want to reset my password using a link sent to my email, so that I can regain access to my account if I forgot my password."

**Feature: Password Reset**

**Scenario: Successful Password Reset**

  Given a valid user account exists with a registered email address
  And a valid reset link is sent to the user's email address
  When the user clicks on the reset link
  Then the user is redirected to a reset page
  And the user is presented with a form to enter a 